# Recursive Neural Networks

Sentence representation
- Bag of words
    - order is lost
- Recurrent Neural Networks
    - sequences of words
    - hard to process long sentences
- Tree!
    - What does the input data look like?
    - How to transform it into a tree?
    
Recursive Networks
- Plain recursive network
$$Linear: h' = f(W^Th + b)$$
- Recursive Neural Tensor Network
$$Quadratic: h' = f(h^T Ah + W^T h + b)$$

## Sentences as trees

- Consider part-of-speech tags (N, V, D, ...)
- Sentences are hierarchycal, subject, verb, verb phrase, noun phrase, etc. 
- Sentiment analysis
    - Each node in the tree will have a label to indicate sentiment
    - This helps with negation problem

- Building a tree is also a ML problem. 
- We'll work with binary trees, although trees can have multiple children. 
    - Always 2 children or 0, never single child.

- Sentiment Analysis
    - Each node has a label for sentiment
    - Only leaves of the tree are actual words

Data: https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip

## Recursive Neural Networks

- Tree Neural Networks (TNN)
- For each leaf node with a word, let $w_i$ be the word embedding for that word.

### Binary trees
- For each inner node we define: $h_i = f(W_{left} x_{left} + W_{right} x_{right} + b)$
    - Let $D$ be the embedding size of words.
    - Weight matrices $W$ should be of size $D x D$.
    - Biases $b$ should be of size $D$.
    - $f$ = relu, tanh, etc. 

- Given a label $y$ and an inner/root node $h$, calculating $p(y | h) = softmax(W_o h + b_o)$ 
    - $W_o$ weight matrix
    - $b_o$ bias term

### N-ary trees

- For each inner node we define: $h = f(\sum_i W(rel(h, i)) x_i + b)$
    - Let $D$ be the embedding size of words.
    - Weight matrices $W$ should be of size $R x D x D$.
    - $R$ is the number of possible parent-child relationships
    - $rel(h, i) -> \{1, ..., R\}$ tells the type of of relationship
    - Biases $b$ should be of size $D$.
    - $f$ = relu, tanh, etc. 

- Given a label $y$ and an inner/root node $h$, calculating $p(y | h) = softmax(W_o h + b_o)$ 
    - $W_o$ weight matrix
    - $b_o$ bias term

In [46]:
import sys
import os
sys.path.append(os.path.abspath('../vendor/machine_learning_examples/'))

In [ ]:
%run -i '../vendor/machine_learning_examples/nlp_class2/recursive_tensorflow.py'

Compiling ops
Instructions for updating:
Use `tf.global_variables_initializer` instead.


- This solution is not idea.
- Each sentence is a different graph, different costs, different gradients, etc. 

## Trees to Sequences

- TNNs - trees
- Recurrent NNs - sequences
- Convert tree to sequence

- Convert the tree into two arrays: 
    - Elements array: postorder traversal of nodes
    - Parents array: for each node at the position `i` in the elements array, parents array in the position `i` contains the id of its parent. 
    - For N-ary trees, create Relations array: where each position `i` contains the type of relation of node `i` with its parent.
    - Word array: each position `i` contains a word if node `i` is a leaf of the graph. 

In [1]:
%run -i '../vendor/machine_learning_examples/nlp_class2/recursive_theano.py'

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


vocab size: 18647
i: 0 cost: 5018.125420100336 correct rate: 0.5218208092485549 time for epoch: 0:08:00.790750
i: 1 cost: 4935.90714544377 correct rate: 0.522543352601156 time for epoch: 0:08:53.519088
i: 2 cost: 4887.338247981783 correct rate: 0.5296242774566474 time for epoch: 0:08:54.141769
i: 3 cost: 4855.987570344421 correct rate: 0.5382947976878613 time for epoch: 0:11:28.104376
i: 4 cost: 4834.864995274912 correct rate: 0.5449421965317919 time for epoch: 0:09:01.347198
i: 5 cost: 4819.428320187071 correct rate: 0.5491329479768786 time for epoch: 0:08:50.154939
i: 6 cost: 4807.048681661282 correct rate: 0.557514450867052 time for epoch: 0:07:48.398398
i: 7 cost: 4794.498076083394 correct rate: 0.5635838150289018 time for epoch: 0:06:57.857527
i: 8 cost: 4780.833737315014 correct rate: 0.5686416184971098 time for epoch: 0:06:59.001770
i: 9 cost: 4760.658218977364 correct rate: 0.5878612716763005 time for epoch: 0:06:58.997463
i: 10 cost: 4670.462795350324 correct rate: 0.623265895

<Figure size 640x480 with 1 Axes>

## Recursive Neural Tensor Networks

- RNTN
- Assumes working with Binary Tree
- Concatenate together the inputs form children $x_{left}$ and $x_{right}$. 
    - $h = f(W^Tx + b)$
    - $x$ now is of size $2D$
    - $W$ of size $2D x D$
- Adding a quadratic element, the inner node formula is for an index $j$: 
    - $h_j = f(x^T A_j x + W_j^T x + b_j)$ 
        - Each element is an scalar. 
    - $A_j$ is of size $2D x 2D$.
    - Since $j = 1..D$, then $A$ is of size $D x 2D x 2D$
- $h(x) = x^T A x + B^T x + c$, quadratic discriminant analysis (2 Gaussians with different covariance)

### Implementation details

- Recursive neural network was: 
$$h(x_L, x_R) = f(W_L^T x_L + W_R^T x_R + b)$$
- Adding quadratic weights: 
$$h(x_L, x_R) = f(x_L^T A_{LL} x_L + x_L^T A_{LR} x_R + x_R^T A_{RR} x_R + W_L^T x_L + W_R^T x_R + b$$
- No more need of relations and parents arrays
- Three lists instead (filled with -1 if null): 
    - words
    - left_children
    - right_children

### RNTN Tensorflow tips

- A custom solution is needed for RNTNs, RNN cells like LSTM or GRU doesn't work. 
- Four sequences to loop simultaneously: 
    - words: contains the word for the current node
    - left_children: contains the index for the left child
    - right_children: contains the index for the right child
    - labels: contains the sentiment for the current node

- Iterating throught these sequences depend on the actual data that we haven't provided yet. So TF has a useful function `while_loop` to allow defining the graph structure iterating over future data. 

```
outputs = tf.while_loop(
    condition, 
    body,  # set state, return output
    initial_values
)
```

- Condition: 
    - `tf.less(n, tf.shape(words)[0])`
    
- Body (only binary tree): 
    - (pseudocode)
    ```
    if current node is leaf: 
        hidden value = embedding[current word]
    else: 
        hidden value = tanh(h_l.dot(W_l) + h_r.dot(W_r) + b)
    ```
    - (tensorflow)
    ```
    h_n = tf.cond(
        current_word >= 0, # is leaf?
        lambda: embedding[current_word],
        lambda: <recursive_calculation>
    )
    ```
- Update hidden nodes: 
    ```
    hiddens = hiddens.write(n, h_n) # hiddens[n] = h_n
    ```
    - What is `hiddens` a `placeholder` or `Variable`?
        - Neither, it is an intermediate output. 
        - It is a `TensorArray`: https://www.tensorflow.org/api_docs/python/tf/TensorArray
            - `read()`
            - `write()`
 

#### What to train on?

Two options: 
- Only label at root node
```
logits = tf.matmul(hiddens.stack(), Wo) + bo
cost_op = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits[-1],       # -1 references to root node (due to postorder)
        labels=labels[-1]
    )
)
```
- Labels at all nodes
    - Sentiment from 0..4 bucketed into: 
        - Positive: 3,4
        - Neutral: 2
        - Negative: 0,1
    - Remove neutral sentences
        - Also from inner nodes
    ```
    labeled_indices = tf.where(labels >= 0)
    cost_op = tf.reduce_mean(...
        logits = tf.gather(logits, labeled_indices),
        labels = tf.gather(labels, labeled_indices), ... )    
    ```